In [25]:
import numpy as np
import pandas as pd
import collections
import statistics
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as st
import scipy
import math
from collections import Counter
import ipywidgets
import seaborn as sns

In [26]:
i_spocib = ipywidgets.Dropdown(options = ["From Console","From File"])
i_n = ipywidgets.IntText(value=20)
i_start = ipywidgets.IntText(value=0)
i_end = ipywidgets.IntText(value=15)
i_on1 = ipywidgets.Checkbox(description='Taks1: Variation series')
i_on2 = ipywidgets.Checkbox(description='Taks2: ...')
i_on3 = ipywidgets.Checkbox(description='Taks3: ...')
i_on4 = ipywidgets.Checkbox(description='Taks4: ...')
i_on5 = ipywidgets.Checkbox(description='Taks5: ...')
i_onti = ipywidgets.Checkbox(description='Taks5 КВАНТИЛІ: ...')
i_q = ipywidgets.IntText(value=100)
i_p = ipywidgets.IntText(value=50)
i_value = ipywidgets.Dropdown(options = ["Disckrete","Сontinuous"])

In [27]:
def console(n,start,end):
    global seriesD
    seriesD= np.random.randint(start,end,n)
    print(f"Вибірка: {list(np.array(seriesD))}")
def Ctask1(on):
    global var_seriesD
    var_seriesD= sorted(list(seriesD))
    print(f"Варіаційний ряд: {var_seriesD}")
def Ctask2(on):
    global elementsD
    elementsD= []
    for x in var_seriesD:
        if x not in elementsD:
            elementsD.append(x)

    global dicttD
    dicttD= {}
    for elem in elementsD:
        count = 0
        for serie in var_seriesD:
            if(int(elem)==serie):
                count+=1
        dicttD[elem]=count

    global stat_rozpidolD
    stat_rozpidolD=pd.DataFrame.from_dict(dicttD,orient='index',columns=['n'],dtype=int)
    stat_rozpidolD.index.name = 'x'
    return stat_rozpidolD.T
def Ctask3(on):
    plt.subplot(111)
    plt.stem(list(dicttD.keys()),list(dicttD.values()),'b--')
    plt.plot( list(dicttD.keys()), list(dicttD.values()),'o-');
    plt.xlabel("x")
    plt.ylabel('n')
    plt.ylim(0)
    plt.show()

    plt.subplot(111)
    plt.plot( list(dicttD.keys()), list(dicttD.values()),'o-');
    plt.xlabel("x")
    plt.ylabel('n')
    plt.ylim(0)
    plt.show()

def Ctask4(on):
    #4.	побудувати емпіричну функцію розподілу;
    fig, axs = plt.subplots(nrows=1, ncols=1, constrained_layout=True, figsize=(12,9))

    global tableD
    tableD = pd.DataFrame(dicttD,index=["m"])
    def draw_empirical(ax, xi, mi):
        ndraw = len(mi) + 1
        n = mi.sum()
        curr = 0
        diff = int(xi.max() - xi.min())
        yticks = np.zeros(ndraw)
        yticks[0] = 0
        yticks[-1] = 1

        ax.plot(np.linspace(xi[0]-diff*0.5, xi[0], diff), np.full(diff, 0.0), color="blue")
        ax.errorbar(xi[0], 0.0, fmt="b>")

        for i in range(len(xi)-1):
            curr += mi[i] / n
            yticks[i+1] = np.round(curr,3)
            ax.plot(np.linspace(xi[i], xi[i+1], diff), np.full(diff, curr), color="blue")
            ax.errorbar(xi[i+1], curr, fmt="b>")

        ax.plot(np.linspace(xi[-1]+diff*0.5, xi[-1], diff), np.full(diff, 1.0), color="blue", label="F(x)")
        ax.legend()
        ax.set_yticks(yticks)
        ax.set_yticklabels(yticks)
        ax.grid(True)
    draw_empirical(axs, tableD.columns.values, tableD.loc["m"].values)
def Ctask5(on):
    #5.	обчислити всі числові характеристики.

    sum_score = sum(var_seriesD)
    num_score = len(var_seriesD)
    avg_score = sum_score/num_score

    most_common,num_most_common = Counter(var_seriesD).most_common(1)[0]
    mode=[]
    for i in var_seriesD:
        if(var_seriesD.count(i)==num_most_common):
            if(i not in mode):
                mode.append(i)
    if(len(mode)>1):
        moda = str(mode)
    else:
        moda = statistics.mode(var_seriesD)

    def Kvantil(a):
        index=len(var_seriesD)*a/100
        if index%1==0:
            return series[int(index)-1]
        else:
            return "---"

    print(f"Середнє вибіркове: {avg_score}")
    print(f"Медіана : {statistics.median(var_seriesD)}")
    print(f"Мода : {moda}")

    print(f"Розмах : {np.max(var_seriesD)-np.min(var_seriesD)}")
    print(f"Девіація : {statistics.variance(var_seriesD)*(len(var_seriesD)-1)}")
    print(f"Варіанса : {statistics.variance(var_seriesD)}")
    print(f"Варіація : {statistics.stdev(var_seriesD)/avg_score}")
    print(f"Стандарт(флуктуація) : {statistics.stdev(var_seriesD)}")

    print(f"Квартилі : 25% = {np.percentile(var_seriesD, 25)}; 50% = {np.percentile(var_seriesD, 50)}; 75% = {np.percentile(var_seriesD, 75)}")

    print(f"Децелі : {list(np.percentile(var_seriesD, np.arange(0, 100, 10)))}")
    print(f"Коефіцієнт Асиметрії  : {st.skew(var_seriesD)}")
    print(f"Коефіцієнт Ексцесу  : {st.kurtosis(var_seriesD)}")
    print(f"Інтерквартильна широта  : {np.percentile(var_seriesD, 75)-np.percentile(var_seriesD, 25)}")
    print(f"Інтердецильна широта  : {np.percentile(var_seriesD, 90)-np.percentile(var_seriesD, 10)}")
def Ctask5Inter(on,q,p):
    def Kvantil(a):
        index=len(series)*a/100
        if index%1==0:
            return seriesD[int(index)-1]
        else:
            return "---"
    if(q>p):
        try:
            print(f"Інтерквантильна широта  : {Kvantil(q)-Kvantil(p)}")
        except Exception:
            print("Не існує даного квантиля!")
    else:
        print("1-ий квантиль повинен бути більшим за 2-ий квантиль")

In [28]:
def Ftask1(on):
    global from_fileFD
    from_fileFD= pd.read_csv("ChastotnaTable.txt",index_col=0)
    lk = list(from_fileFD.keys())
    lv = from_fileFD.values
    global keysFD
    keysFD= []
    global valuesFD
    valuesFD= []
    lv.tolist()
    for i in lv:
        for j in range(len(i)):
            valuesFD.append(i[j])
    for i in lk:
        keysFD.append(int(i))
    lettersFD =[]
    letterFD = ''
    global seriesFD
    seriesFD = []
    for i in range(len(keysFD)):
        letterFD = str(keysFD[i])*valuesFD[i]
        lettersFD.append(letterFD)
    strr=''.join(lettersFD)

    for i in strr:
        seriesFD.append(int(i))
    global var_seriesFD
    var_seriesFD= sorted(list(seriesFD))
    print(f"Варіаційний ряд: {var_seriesFD}")
def Ftask2(on):
    global from_fileFD1
    from_fileFD1= pd.read_csv("ChastotnaTable.txt",index_col=0)
    return from_fileFD1
def Ftask3(on):
    #3.	представити графічно статистичний матеріал,
    plt.subplot(111)
    plt.stem(keysFD,valuesFD,'b--')
    plt.plot( keysFD, valuesFD,'o-');
    plt.xlabel("x")
    plt.ylabel('n')
    plt.ylim(0)
    plt.show()

    plt.subplot(111)
    plt.plot( keysFD, valuesFD,'o-');
    plt.xlabel("x")
    plt.ylabel('n')
    plt.ylim(0)
    plt.show()

def Ftask4(on):
    #4.	побудувати емпіричну функцію розподілу;
    dicttFD = {}
    for key in keysFD:
        for value in valuesFD:
            dicttFD[key]=value
    
    fig, axs = plt.subplots(nrows=1, ncols=1, constrained_layout=True, figsize=(12,9))

    tableFD = pd.DataFrame(dicttFD,index=["ident"])
    def draw_empirical(ax, xi, mi):
        ndraw = len(mi) + 1
        n = mi.sum()
        curr = 0
        diff = int(xi.max() - xi.min())
        yticks = np.zeros(ndraw)
        yticks[0] = 0
        yticks[-1] = 1

        ax.plot(np.linspace(xi[0]-diff*0.5, xi[0], diff), np.full(diff, 0.0), color="blue")
        ax.errorbar(xi[0], 0.0, fmt="b>")

        for i in range(len(xi)-1):
            curr += mi[i] / n
            yticks[i+1] = np.round(curr,3)
            ax.plot(np.linspace(xi[i], xi[i+1], diff), np.full(diff, curr), color="blue")
            ax.errorbar(xi[i+1], curr, fmt="b>")

        ax.plot(np.linspace(xi[-1]+diff*0.5, xi[-1], diff), np.full(diff, 1.0), color="blue", label="F(x)")
        ax.legend()
        ax.set_yticks(yticks)
        ax.set_yticklabels(yticks)
        ax.grid(True)
        plt.suptitle("Графік емпіричної функції")
    draw_empirical(axs, tableFD.columns.values, tableFD.loc["ident"].values)
    
def Ftask5(on):
    #5.	обчислити всі числові характеристики.

    sum_score = sum(var_seriesFD)
    num_score = len(var_seriesFD)
    avg_score = sum_score/num_score

    most_common,num_most_common = Counter(var_seriesFD).most_common(1)[0]
    mode=[]
    for i in var_seriesFD:
        if(var_seriesFD.count(i)==num_most_common):
            if(i not in mode):
                mode.append(i)
    if(len(mode)>1):
        moda = str(mode)
    else:
        moda = statistics.mode(var_seriesFD)

    def Kvantil(a):
        index=len(var_seriesFD)*a/100
        if index%1==0:
            return var_seriesFD[int(index)-1]
        else:
            return "---"

    print(f"Середнє вибіркове: {avg_score}")
    print(f"Медіана : {statistics.median(var_seriesFD)}")
    print(f"Мода : {moda}")

    print(f"Розмах : {np.max(var_seriesFD)-np.min(var_seriesFD)}")
    print(f"Девіація : {statistics.variance(var_seriesFD)*(len(var_seriesFD)-1)}")
    print(f"Варіанса : {statistics.variance(var_seriesFD)}")
    print(f"Варіація : {statistics.stdev(var_seriesFD)/avg_score}")
    print(f"Стандарт(флуктуація) : {statistics.stdev(var_seriesFD)}")

    print(f"Квартилі : 25% = {np.percentile(var_seriesFD, 25)}; 50% = {np.percentile(var_seriesFD, 50)}; 75% = {np.percentile(var_seriesFD, 75)}")

    print(f"Децелі : {list(np.percentile(var_seriesFD, np.arange(0, 100, 10)))}")
    print(f"Коефіцієнт Асиметрії  : {st.skew(var_seriesFD)}")
    print(f"Коефіцієнт Ексцесу  : {st.kurtosis(var_seriesFD)}")
    print(f"Інтерквартильна широта  : {np.percentile(var_seriesFD, 75)-np.percentile(var_seriesFD, 25)}")
    print(f"Інтердецильна широта  : {np.percentile(var_seriesFD, 90)-np.percentile(var_seriesFD, 10)}")
def Ftask5Inter(on,q,p):
    def Kvantil(a):
        index=len(var_seriesFD)*a/100
        if index%1==0:
            return var_seriesFD[int(index)-1]
        else:
            return "---"
    if(q>p):
        try:
            print(f"Інтерквантильна широта  : {Kvantil(q)-Kvantil(p)}")
        except Exception:
            print("Не існує даного квантиля!")
    else:
        print("1-ий квантиль повинен бути більшим за 2-ий квантиль")

In [29]:
def Cconsole(n,start,end):
    global seriesCC
    seriesCC= np.random.uniform(start,end,n)
    print(f"Вибірка: {list(np.array(seriesCC))}")
def CCtask1(on):
    global var_seriesCC
    var_seriesCC= sorted(list(seriesCC))
    print(f"Варіаційний ряд: {var_seriesCC}")
def CCtask2(on):
    global rCC 
    rCC = 0
    for i in range(0,100):
        if(2**i<len(seriesCC)<=2**(i+1)):
            rCC = i
    global rozmahCC
    rozmahCC = max(seriesCC)-min(seriesCC)
    global count_intervalsCC
    count_intervalsCC = rCC+1
    global stepCC
    stepCC= rozmahCC/(count_intervalsCC)
    global timesCC 
    timesCC = []
    global startCC
    startCC = min(seriesCC)
    global stopCC 
    stopCC = max(seriesCC)
    timesCC.append(startCC)

    while startCC < stopCC:
        startCC += stepCC
        timesCC.append(startCC)
    global intervalsCC 
    intervalsCC=[]
    for item in range(len(timesCC)):
        interval = []
        try:
            interval.append(timesCC[item])
            interval.append(timesCC[item+1])
            intervalsCC.append(interval)
        except Exception:
            pass
    def count(list1, l, r): 
        c = 0 
        for x in list1: 
            if x>= l and x<= r: 
                c+= 1 
        return c 
    global dicttCC 
    dicttCC= {}
    for i in range(len(intervalsCC)):    
        dicttCC[str(intervalsCC[i])]=count(var_seriesCC,intervalsCC[i][0],intervalsCC[i][1])
    global discrete_xCC 
    discrete_xCC = []
    for i in range(len(intervalsCC)):
        discrete_xCC.append((intervalsCC[i][1]+intervalsCC[i][0])/2)
    global stat_rozpidolCC
    stat_rozpidolCC=pd.DataFrame.from_dict(dicttCC,orient='index',columns=['n'],dtype=int)
    stat_rozpidolCC.index.name = 'x'
    stat_rozpidolCC['Dx'] = discrete_xCC
    
    return stat_rozpidolCC.T    
    
def CCtask3(on):
    #3.	представити графічно статистичний матеріал,
    plt.subplot(111)
    plt.stem(discrete_xCC,list(dicttCC.values()),'b--')
    plt.plot( discrete_xCC, list(dicttCC.values()),'o-');
    plt.xlabel("x")
    plt.ylabel('n')
    plt.ylim(0)
    plt.show()

    plt.subplot(111)
    plt.plot( discrete_xCC, list(dicttCC.values()),'o-');
    plt.xlabel("x")
    plt.ylabel('n')
    plt.ylim(0)
    plt.show()


    
def CCtask4(on):
    #4.	побудувати емпіричну функцію розподілу;
    norm_cdf = scipy.stats.norm.cdf(var_seriesCC)
    sns.lineplot(x=var_seriesCC, y=norm_cdf)
    plt.show()
  

    
def CCtask5(on):
    #5.	обчислити всі числові характеристики.

    sum_score = sum(var_seriesCC)
    num_score = len(var_seriesCC)
    avg_score = sum_score/num_score

    most_common,num_most_common = Counter(var_seriesCC).most_common(1)[0]
    mode=[]
    for i in var_seriesCC:
        if(var_seriesCC.count(i)==num_most_common):
            if(i not in mode):
                mode.append(i)
    if(len(mode)>1):
        moda = str(mode)
    else:
        moda = statistics.mode(var_seriesCC)

    def Kvantil(a):
        index=len(seriesCC)*a/100
        if index%1==0:
            return seriesCC[int(index)-1]
        else:
            return "---"


    print(f"Середнє вибіркове: {avg_score}")
    print(f"Медіана : {statistics.median(seriesCC)}")
    print(f"Мода : {moda}")

    print(f"Розмах : {np.max(var_seriesCC)-np.min(var_seriesCC)}")
    print(f"Девіація : {statistics.variance(var_seriesCC)*(len(var_seriesCC)-1)}")
    print(f"Варіанса : {statistics.variance(var_seriesCC)}")
    print(f"Варіація : {statistics.stdev(var_seriesCC)/avg_score}")
    print(f"Стандарт(флуктуація) : {statistics.stdev(var_seriesCC)}")

    print(f"Квартилі : 25% = {np.percentile(var_seriesCC, 25)}; 50% = {np.percentile(var_seriesCC, 50)}; 75% = {np.percentile(var_seriesCC, 75)}")

    print(f"Децелі : {list(np.percentile(var_seriesCC, np.arange(0, 100, 10)))}")
    print(f"Коефіцієнт Асиметрії  : {st.skew(seriesCC)}")
    print(f"Коефіцієнт Ексцесу  : {st.kurtosis(seriesCC)}")
    print(f"Інтерквартильна широта  : {np.percentile(var_seriesCC, 75)-np.percentile(var_seriesCC, 25)}")
    print(f"Інтердецильна широта  : {np.percentile(var_seriesCC, 90)-np.percentile(var_seriesCC, 10)}")

def CCtask5Inter(on,q,p):
    def Kvantil(a):
        index=len(series)*a/100
        if index%1==0:
            return series[int(index)-1]
        else:
            return "---"
    if(q>p):
        try:
            print(f"Інтерквантильна широта  : {Kvantil(q)-Kvantil(p)}")
        except Exception:
            print("Не існує даного квантиля!")
    else:
        print("1-ий квантиль повинен бути більшим за 2-ий квантиль")

def CFtask1(on):
    global from_file
    from_file= pd.read_csv("ChastotnaTableNeperervna.txt",index_col=0)
    global lk
    lk = list(from_file.keys())
    global lv
    lv = from_file.values
    global intervals
    intervals =[]
    global keys
    keys = []
    global values
    values = []
    lv.tolist()
    for i in lv:
        for j in range(len(i)):
            values.append(i[j])
    for i in lk:
        interval =[]
        s = i.split(';')
        interval.append(float(s[0]))
        interval.append(float(s[1]))
        intervals.append(interval)
    global count_intervals 
    count_intervals= 0
    for i in intervals:
        count_intervals+=1
    
    from_file= pd.read_csv("ChastotnaTableNeperervna.txt")
    from_file = from_file.T
    global discrete_x 
    discrete_x= ["DX"]
    for i in range(len(intervals)):
        discrete_x.append((intervals[i][1]+intervals[i][0])/2)
    from_file["DX"]=discrete_x
    from_file = from_file.T
    from_file.set_index('x')
    from_file = from_file.T
    keys = from_file['DX'].values
    keys = keys.tolist()
    
    values= from_file.values
    values = values.tolist()
    global vallues 
    vallues= []
    for i in values:
        try:
            vallues.append(int(i[0]))
        except Exception:
            1+1
    keys.remove('DX')
    global letters 
    letters=[]
    global letter 
    letter= ''
    global series 
    series= []

    for i in range(len(keys)):
        letter = (str(keys[i])+' ')*vallues[i]
        s=letter.split(' ')
        s.remove('')
        letters.append(s)

     
    series= []
    
    s=''
    for item in letters:
        for j in range(len(item)):
            series.append(float(item[j]))

    global var_series 
    var_series= sorted(list(series))
    print(f"Варіаційний ряд: {var_series}")

    
def CFtask2(on):
    return from_file.T.set_index('x')
def CFtask3(on):
    #3.	представити графічно статистичний матеріал,
    plt.subplot(111)
    plt.stem(keys,vallues,'b--')
    plt.plot( keys, vallues,'o-');
    plt.xlabel("x")
    plt.ylabel('n')
    plt.ylim(0)
    plt.show()

    plt.subplot(111)
    plt.plot( keys, vallues,'o-');
    plt.xlabel("x")
    plt.ylabel('n')
    plt.ylim(0)
    plt.show()


def CFtask4(on):
    #4.	побудувати емпіричну функцію розподілу;
    norm_cdf = scipy.stats.norm.cdf(var_seriesCC)
    sns.lineplot(x=var_seriesCC, y=norm_cdf)
    plt.show()

       
def CFtask5(on):
    #5.	обчислити всі числові характеристики.

    sum_score = sum(var_series)
    num_score = len(var_series)
    avg_score = sum_score/num_score

    most_common,num_most_common = Counter(var_series).most_common(1)[0]
    mode=[]
    for i in var_series:
        if(var_series.count(i)==num_most_common):
            if(i not in mode):
                mode.append(i)
    if(len(mode)>1):
        moda = str(mode)
    else:
        moda = statistics.mode(var_series)

    def Kvantil(a):
        index=len(series)*a/100
        if index%1==0:
            return series[int(index)-1]
        else:
            return "---"


    print(f"Середнє вибіркове: {avg_score}")
    print(f"Медіана : {statistics.median(series)}")
    print(f"Мода : {moda}")

    print(f"Розмах : {np.max(series)-np.min(series)}")
    print(f"Девіація : {statistics.variance(series)*(len(series)-1)}")
    print(f"Варіанса : {statistics.variance(series)}")
    print(f"Варіація : {statistics.stdev(series)/avg_score}")
    print(f"Стандарт(флуктуація) : {statistics.stdev(series)}")

    print(f"Квартилі : 25% = {np.percentile(var_series, 25)}; 50% = {np.percentile(var_series, 50)}; 75% = {np.percentile(var_series, 75)}")

    print(f"Децелі : {list(np.percentile(var_series, np.arange(0, 100, 10)))}")
    print(f"Коефіцієнт Асиметрії  : {st.skew(series)}")
    print(f"Коефіцієнт Ексцесу  : {st.kurtosis(series)}")
    print(f"Інтерквартильна широта  : {np.percentile(var_series, 75)-np.percentile(var_series, 25)}")
    print(f"Інтердецильна широта  : {np.percentile(var_series, 90)-np.percentile(var_series, 10)}")
def CFtask5Inter(on,q,p):
    def Kvantil(a):
        index=len(series)*a/100
        if index%1==0:
            return series[int(index)-1]
        else:
            return "---"
    if(q>p):
        try:
            print(f"Інтерквантильна широта  : {Kvantil(q)-Kvantil(p)}")
        except Exception:
            print("Не існує даного квантиля!")
    else:
        print("1-ий квантиль повинен бути більшим за 2-ий квантиль")

In [30]:
def choose1(spocib):
    if(spocib=="From Console"):
        ipywidgets.interact_manual(console,n=i_n,start=i_start,end=i_end)
        ipywidgets.interact_manual(Ctask1,on=i_on1)
        ipywidgets.interact_manual(Ctask2,on=i_on2)
        ipywidgets.interact_manual(Ctask3,on=i_on3)
        ipywidgets.interact_manual(Ctask4,on=i_on4)
        ipywidgets.interact_manual(Ctask5,on=i_on5)
        ipywidgets.interact_manual(Ctask5Inter,on=i_onti,q = i_q,p = i_p)
    elif(spocib=="From File"):
        ipywidgets.interact_manual(Ftask1,on=i_on1)
        ipywidgets.interact_manual(Ftask2,on=i_on2)
        ipywidgets.interact_manual(Ftask3,on=i_on3)
        ipywidgets.interact_manual(Ftask4,on=i_on4)
        ipywidgets.interact_manual(Ftask5,on=i_on5)
        ipywidgets.interact_manual(Ftask5Inter,on=i_onti,q = i_q,p = i_p)

def choose2(spocib):
    if(spocib=="From Console"):
        ipywidgets.interact_manual(Cconsole,n=i_n,start=i_start,end=i_end)
        ipywidgets.interact_manual(CCtask1,on=i_on1)
        ipywidgets.interact_manual(CCtask2,on=i_on2)
        ipywidgets.interact_manual(CCtask3,on=i_on3)
        ipywidgets.interact_manual(CCtask4,on=i_on4)
        ipywidgets.interact_manual(CCtask5,on=i_on5)
        ipywidgets.interact_manual(CCtask5Inter,on=i_onti,q = i_q,p = i_p)
    elif(spocib=="From File"):
        ipywidgets.interact_manual(CFtask1,on=i_on1)
        ipywidgets.interact_manual(CFtask2,on=i_on2)
        ipywidgets.interact_manual(CFtask3,on=i_on3)
        ipywidgets.interact_manual(CFtask4,on=i_on4)
        ipywidgets.interact_manual(CFtask5,on=i_on5)
        ipywidgets.interact_manual(CFtask5Inter,on=i_onti,q = i_q,p = i_p)


In [31]:
def typeValue(value):
    if(value == 'Disckrete'):
        spocib = ipywidgets.interact_manual(choose1,spocib=i_spocib)
        if(spocib=="From Console"):
            ipywidgets.interact_manual(console,n=i_n,start=i_start,end=i_end)
            ipywidgets.interact_manual(Ctask1,on=i_on1)
            ipywidgets.interact_manual(Ctask2,on=i_on2)
            ipywidgets.interact_manual(Ctask3,on=i_on3)
            ipywidgets.interact_manual(Ctask4,on=i_on4)
            ipywidgets.interact_manual(Ctask5,on=i_on5)
            ipywidgets.interact_manual(Ctask5Inter,on=i_on5,q = i_q,p = i_p)
        elif(spocib=="From File"):
            ipywidgets.interact_manual(Ftask1,on=i_on1)
            ipywidgets.interact_manual(Ftask2,on=i_on2)
            ipywidgets.interact_manual(Ftask3,on=i_on3)
            ipywidgets.interact_manual(Ftask4,on=i_on4)
            ipywidgets.interact_manual(Ftask5,on=i_on5)
            ipywidgets.interact_manual(Ftask5Inter,on=i_on5,q = i_q,p = i_p)
    elif(value == 'Сontinuous'):
        spocib = ipywidgets.interact_manual(choose2,spocib=i_spocib)
        if(spocib=="From Console"):
            ipywidgets.interact_manual(Cconsole,n=i_n,start=i_start,end=i_end)
            ipywidgets.interact_manual(CCtask1,on=i_on1)
            ipywidgets.interact_manual(CCtask2,on=i_on2)
            ipywidgets.interact_manual(CCtask3,on=i_on3)
            ipywidgets.interact_manual(CCtask4,on=i_on4)
            ipywidgets.interact_manual(CCtask5,on=i_on5)
            ipywidgets.interact_manual(CCtask5Inter,on=i_on5,q = i_q,p = i_p)
        elif(spocib=="From File"):
            ipywidgets.interact_manual(CFtask1,on=i_on1)
            ipywidgets.interact_manual(CFtask2,on=i_on2)
            ipywidgets.interact_manual(CFtask3,on=i_on3)
            ipywidgets.interact_manual(CFtask4,on=i_on4)
            ipywidgets.interact_manual(CFtask5,on=i_on5)
            ipywidgets.interact_manual(CFtask5Inter,on=i_on5,q = i_q,p = i_p)


In [33]:
ipywidgets.interact_manual(typeValue,value=i_value);

interactive(children=(Dropdown(description='value', index=1, options=('Disckrete', 'Сontinuous'), value='Сonti…